# Homework05

Some exercises with image and audio data preparation.

## Goals

- Even more practice with lists
- Get familiar with pandas `DataFrames`
- Practice dataset exploration and normalization/scaling
- Set up a dataset for proper classification

## Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework

In [ ]:
!wget -q https://github.com/PSAM-5020-2026S-A/5020-utils/raw/main/src/audio_utils.py
!wget -q https://github.com/PSAM-5020-2026S-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2026S-A/5020-utils/raw/main/src/image_utils.py

!wget -q https://github.com/PSAM-5020-2026S-A/Homework03/raw/main/Homework03_utils.pyc
!wget -q https://github.com/PSAM-5020-2026S-A/Homework04/raw/main/Homework04_utils.pyc

!wget -qO- https://github.com/PSAM-5020-2026S-A/5020-utils/releases/latest/download/forest-tree.tar.gz | tar xz
!wget -qO- https://github.com/PSAM-5020-2026S-A/5020-utils/releases/latest/download/instruments.tar.gz | tar xz

!wget -q https://github.com/PSAM-5020-2026S-A/5020-utils/raw/refs/heads/main/datasets/audio/hood-internet.tar.gz
!tar -xzf ./hood-internet.tar.gz -C ./data/audio/ && rm -rf hood-internet.tar.gz

In [ ]:
import librosa
import matplotlib.pyplot as plt
import pandas as pd
import random

from IPython.display import Audio
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from os import listdir

from image_utils import make_image, get_pixels

from Homework05_utils import AwesomeAudioClassifier, AwesomeImageClassifier

AUDIO_PATH = "./data/audio/instruments/test"
IMAGE_PATH = "./data/image/forest-tree/test"

## More Image/Audio Classification

We're going to re-visit the classification exercises from `Homework04` and `Homework05`.

This exercise is a bit different though. In some ways it's the opposite of the previous exercises because we'll already have classification models ready to be used, but will have to normalize and standardize our dataset in order to run them. This is more representative of the type of work that goes into using real, pre-trained, ML models in the wild.

### The Models

We have two `Awesome` models, one for audio classification (`AwesomeAudioClassifier`), and one for image classification (`AwesomeImageClassifier`).

Unlike the classification models we set up for `Homework04` and `Homework05`, these models have more strict requirements about the shape and values of their input data. We can't run them on the files as they are.

### The Data

Audio and Image files are in the `data/audio/` and `data/image/` directories respectively.

We will use the `get_training_data()` from each of our classifiers to get the initial training data and labels for our audio and image files.

### The Features

This is the challenging part.

The data returned by `get_training_data()` is a representation of the content of the audio and image files, but it hasn't been processed or normalized in order to be used by the classifier models provided.

We can try to create a `DataFrame` directly from those, and it might seem like it works, but if we take a look at the result we'll see some `NaN` (Not-a-Number) values in some of the columns, and if we send that to the model it will barf and complain about having `NaN`s in the data.

This happens because all of the audios and images have different sizes. Hoooray !!

Welcome to Machine Learning. This is probably where most of the time in any ML project is spent: cleaning up data and making sure it has the right format, size and shape that a model expects.

For this exercise it won't be too hard to fix these.

Let's start with the audio files since they're one-dimensional, and once we have the audio modeling working we'll come back to the image files.

## Audio Data

Let's run `AwesomeAudioClassifier.get_training_data()` function to get some audio data. This function returns audio data and labels from files inside a specified directory.

In [ ]:
features, labels = AwesomeAudioClassifier.get_training_data(AUDIO_PATH)

### Audio Features

The audio data returned is actually in the frequency domain and is not samples, so even though we can't play these audio files, we can still plot this data and will have to normalize and clean it before we can run it through our classifier.

Let's take a look at this data.

What are the labels ? How many records do we have ? How many features do we have in each record ? Can we plot our data ?

In [ ]:
# TODO: How many records ?

# TODO: How many features ?

# TODO: Plot some features

### Looks like data !

Looks like audio frequency-domain data to be more specific.

If we were to follow some of the data exploration steps we saw in class we would want to put this data in a `DataFrame` in order to get calculate some of its statistical properties, and maybe scale/normalize it before we use it in a classifier model.

Let's try it:

In [ ]:
features_df = pd.DataFrame(features)
features_df

It looks like it works, but when we look closely at the `DataFrame`, specially if we look at the features that are further to the right, we'll see our problem: `NaN` values.

As previously mentioned, this happens because the length of our features is different for each file.

### Fix Audio Data

Let's fix this by making all of the feature lists have the same length. We can either pad the short ones or slice the longer ones to have the same length as the shortest feature list. The second option is preferable since padding would require adding information to the dataset and that might have side effects.

So, we'll go through the lists of lists, create a list of lengths and find the smallest length.

Then, we'll iterate through the lists of lists and slice all the feature lists to have the same length.

In [ ]:
# TODO: go through the list of features and make their lengths consistent

A `DataFrame` created using the cropped features should look more consistent now.

In [ ]:
features_df = pd.DataFrame(cropped_features)
features_df

### Bonus: Empty features

We've removed the `NaN` values, but it seems like we have a lot of columns that are all zeros or nearly all zeros, or are full of values that are close to zero, like `1e-7`.

While it's not necessary, we could also remove these in order to speed up the modeling later.

The `.loc[]` accessor of the `DataFrame` can be used to select columns using boolean indexing.

In [ ]:
# sum of all columns
display(features_df.sum())

# columns where the sum is more than 100
display((features_df.sum() > 100))

# TODO: remove columns with no information

### Run the Model

Now that we have a `DataFrame` with consistent rows, we can fit and evaluate our model.

The next cell runs the pre-defined classification model, fitting it with our `features_df` `DataFrame` and then reports the accuracy of our model.

We just have to run it.

In [ ]:
# Fit the classifier and report training accuracy
AwesomeAudioClassifier.fit(features_df, labels)

### Scale / Normalize

Hmmm.... it runs, but we can do better.

We saw in class that normalizing/rescaling our features can help us find actual patterns in our data. It also helps models find patterns.

Try scaling the `DataFrame` using either a `MinMaxScaler` or a `StandardScaler` object.

In [ ]:
# TODO: scale/normalize features

### Run the Model Again

This time with scaled data.

In [ ]:
# Fit the classifier and report training accuracy
AwesomeAudioClassifier.fit(features_scaled_df, labels)

### Interpretation

<span style="color:hotpink;">
Do different scaling strategies influence the prediction results ? What might that tell us about our data ?
</span>

<span style="color:hotpink;">
EDIT THIS CELL WITH ANSWER
</span>

## Image Data

This is a bit trickier, but only because our classifier model for images is a bit pickier. Not only do we have to ensure that all of our records have the same number of features (images have the same number of pixels), we will also have to convert the pixels into grayscale pixels.

Let's start by reading the data and looking at what we get.

In [ ]:
imgs, labels = AwesomeImageClassifier.get_training_data(IMAGE_PATH)

### Image Data

What did we get in the `imgs` variable ? How many records do we have ? How many features does each record/image have ?

In [ ]:
# TODO: look at the imgs and labels variables and get some information about the data

### Create Image Features

It seems like we have actual `PIL` image objects and their labels. 

This will work to our advantage because if we try to just create a `DataFrame` of the extracted pixels from these images we'll probably have a problem with missing feature values again.

In [ ]:
features = []
for img in imgs:
  features.append(get_pixels(img))

print(len(features), len(features[0]), len(features[11]))

features_df = pd.DataFrame(features)
features_df

### Fix Images

We could follow a similar approach to how we fixed the audio data, and just slice our pixel arrays to have the same length as the shortest pixel array, but that will distort our images. Try it out to see the result, but instead of taking pixels out from the end of the image, what we really have to do is change their dimensions so they all have the same `width` and `height` before we get their pixels.

There are a couple of ways to achieve this:
- Crop: use the `image.crop()` function to cut the images.
- Resize: use `image.resize()` to stretch/squeeze the images into specific shapes.

Documentation for [`crop()`](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.crop) and [`resize()`](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.resize).

Take a look at a few images before picking a strategy and then take a look after to see what the chosen strategy does to the images.

In [ ]:
# TODO: look at characteristics/dimensions of the images

# TODO: go through the images and make their dimensions consistent

# TODO: look at some images

### Create Features

Now that we have images with consistent dimensions, we can extract their pixels and convert them to grayscale, so we get a nice looking `DataFrame` to send to our classifier model.

In [ ]:
# TODO: calculate grayscale pixel values

# TODO: look at some images with make_image()

# TODO: create DataFrame

### Run the Image Model

Now that we have a `DataFrame` with consistent features, we can fit and evaluate our model.

The next cell runs the pre-defined classification model, fitting it with our `features_df` `DataFrame` and then reports the accuracy of our model.

We just have to run it (and wait a bit because it can take up to $20$ seconds for it to run).

In [ ]:
# Fit the classifier and report training accuracy
AwesomeImageClassifier.fit(features_df, labels)

### Scaling / Normalizing

Run the classifier model again, but this time using normalized features.

In [ ]:
# TODO: create scaler object, scale data and re-run classification

### Interpretation

<span style="color:hotpink;">
Do different scaling strategies influence the prediction results ? What might that tell us about our data ?
</span>

<span style="color:hotpink;">
EDIT THIS CELL WITH ANSWER
</span>

## Shazam !

Yep. Let's replicate the Shazam algorithm. It's the same type of classification task we've been looking at, but instead of classifying a piece of audio into one of $3$ classes, the algorithm has to pick which of the millions of songs the piece of audio most likely came from.

We've created some simple classification models by manually extracting features that we think are important for the differentiation of our classes. This is ok if we're only dealing with a few classes, but it doesn't scale. If we had to classify audio into hundreds, or thousands, of classes, looking at graphs of relative frequencies and loudness for all classes becomes intractable.

We need something a little more systematic. Something that automatically extracts meaningful numeric information from audio, independent of the type of audio.

Let's look at the frequency domain again, and use the librosa library to help us extract consistent frequency information from audio files.

## Librosa Feature Extraction

Librosa is useful because it has many feature extraction functions to actually get meaningful frequency information from our audios files:

[Librosa Reference and Documentation](https://librosa.org/doc/latest/feature.html)

### Some example functions

- `librosa.feature.spectral_centroid` (extracts average frequency)
- `librosa.feature.spectral_bandwidth` (extracts frequency range)
- `librosa.feature.spectral_rolloff(roll_percent=0.01)` (extarcts smallest frequency)
- `librosa.feature.spectral_rolloff(roll_percent=0.99)` (extracts largest frequency)

Let's load a song and see what running these looks like:

### ❗️❗️ WARNING ❗️❗️

The songs we'll be using are from mixtapes by [The Hood Internet](https://www.thehoodinternet.com/). They were part of a wave of _mashup_ producers in the early $2\text{,}000\text{s}$ that mixed "_classic_" american indie, pop and rap music from other time periods.

We're using their mixtapes here mostly because: they're available freely on the internet, they encompass a good variety of songs and styles, and there are a good number of songs; not too few nor too many.

The lyrics mostly come from the rap songs being sampled and contain references to all the things that rap artists were rapping about in the $90\text{s}$ and $00\text{s}$ (sex, drugs, guns, money...).

We definitely don't have to listen to the songs to complete the exercise.

And, I am looking for suggestions of alternative "datasets" for this exercise.

### Back to Features Extraction

In [ ]:
y,sr = librosa.load("./data/audio/hood-internet/mixtape04/02 David Banner x Fujiya & Miyagi - Get Like Pterodactyls.mp3")

plt.plot(y)
plt.show()

display(Audio(y, rate=sr))

This is how we extract the center frequency and the lowest/highest frequencies of our signal, throughout the song.

Because these functions were written to work with multi-channel audio, they'll always return a list of lists. In our case, we just have a single channel, so our result will be at index $0$ of the returned list.

That's why we have `sc[0]`, `smin[0]`, `smax[0]` below.

In [ ]:
# average frequency
sc = librosa.feature.spectral_centroid(y=y)

# min and max frequencies
smin = librosa.feature.spectral_rolloff(y=y, roll_percent=0.01)
smax = librosa.feature.spectral_rolloff(y=y, roll_percent=0.99)

plt.plot(sc[0])
plt.plot(smin[0])
plt.plot(smax[0])
plt.show()

This is a start.

The functions give us multiple values because the avg/min/max frequency changes over time, but we can average the `spectral_centroid` and the two `spectral_rolloff` frequencies to get $3$ values that represent our entire audio file.

In [ ]:
sc_val = sum(sc[0]) / len(sc[0])
smin_val = sum(smin[0]) / len(smin[0])
smax_val = sum(smax[0]) / len(smax[0])

plt.plot([0, len(sc[0])], [sc_val, sc_val])
plt.plot([0, len(smin[0])], [smin_val, smin_val])
plt.plot([0, len(smax[0])], [smax_val, smax_val])
plt.show()

Not bad.

### MFCCs

But, we can extend this idea with Mel-Frequency Cepstral Coefficients (MFCC). MFCCs are a compact, numerical representation of an audio signal's frequency characteristics, that divide an audio frequency range into bands based on human hearing sensitivities. So, instead of just $3$ frequencies, like we got above, the total frequency range is divided into $10$ to $20$ values that represent our audio signal.

And of course `librosa` has an `mfcc()` function:

In [ ]:
mfccs = librosa.feature.mfcc(y=y, n_mfcc=10)
print(mfccs.shape)

for vs in mfccs:
  plt.plot(vs)
plt.show()

### Average MFCCs

Like the `spectral_centroid()` function, the `mfcc()` function gives us multiple values for each frequency channel/band, since these change over time as the audio changes. We'll assume that one value per frequency band is enough and average them over time, so we end up with only $10$ numbers to represent our audio.

In [ ]:
mfccs_avg = mfccs.mean(axis=1)
print(mfccs_avg.shape)

for vs in mfccs_avg:
  plt.plot([0,100], [vs, vs])
plt.show()

### Back to Shazam

This is what's behind the [Shazam algorithm](https://www.youtube.com/watch?v=b6xeOLjeKs0): a large database of songs, where each song is described with only a few features so they can be organized, searched, queried and compared very very quickly.

Another [video](https://www.youtube.com/watch?v=kMNSAhsyiDg) reference.

Let's implement something like this.

We'll focus on a single album, with about $30$ songs, but the same strategy (with minor modifications) will work for datasets of many more songs.

Here's the outline of what we have to do:

1. Extract MFCC features from all songs in our data base (single album)
2. Create a dataset of song features
3. Given an input sound clip: extract its MFCC features
4. Find closest song in database by feature distance

In [ ]:
# First, let's grab all audio file names from our album directory

TRACKS_DIR = "./data/audio/hood-internet/mixtape04"
fnames = sorted([f for f in listdir(TRACKS_DIR) if f.endswith("mp3")])

In [ ]:
# Now we define a helper function that:
#   given a list of samples, 
#   returns a dictionary of extracted mfcc values

def get_features(y, n_mfcc=20):
  mfeats = {}
  mfccs = librosa.feature.mfcc(y=y, n_mfcc=n_mfcc).mean(axis=1)
  for idx,fcc in enumerate(mfccs):
    mfeats[f"mfcc{idx}"] = float(fcc)
  return mfeats

In [ ]:
# TODO: run the get_features() function in one of the loaded files
# TODO: check the output and see if it makes sense

### 1. Extract MFCC features from all songs

It takes about $20$ seconds to extract features for all songs.

In [ ]:
# TODO: Run the function on all tracks and extract their average MFCCs

# TODO: iterate over the list of filenames, open each file,
#       extract mfcc features and append result to a list

track_feats = []

### 2. Create `DataFrame` of song features

In [ ]:
# TODO: put results in pandas DataFrame using the from_records() function

### 3. Given an input sound clip: extract its MFCC features

In [ ]:
# Let's open track 02 (index 1) and get 5 seconds of audio from the center of the song

# Track 02
fpath = f"{TRACKS_DIR}/{fnames[1]}"
y_in, sr_in = librosa.load(fpath)

# get 5 seconds from middle of song
midx = int(len(y_in) // 2)
half_win = int(2.5 * sr_in)
samples_5sec = y_in[midx - half_win : midx + half_win]

feat_in = [get_features(samples_5sec)]

# And create DataFrame with MFCC features for the 5-second clip
in_df = pd.DataFrame.from_records(feat_in)
in_df

### 4. Find closest song in database by feature distance

Because both our dataset of song features and the features from the input clip we want to find are in `DataFrames`, we can do some `DataFrame` math to quickly calculate the difference between the input clip features and features from all of the songs in our song database. We'll use this to find the row in our dataset that is closest to the input audio.

We can start the process with this line of code:
```py
dists = (tracks_df - in_df.iloc[0]).abs().sum(axis=1)
```

Pretty intense. Let's break it down:

- `tracks_df - in_df.iloc[0]`: this subtracts the first (and only) row of the input `DataFrame` from every row of the song dataset. The result is another `DataFrame` with $34$ rows and $20$ columns, where the cells don't hold MFCC values anymore, but the difference between the values in the input clip and the values from the songs in the dataset.

- `.abs()`: the absolute value of the difference. After the subtraction, some of the values might be negative, but we only care about their magnitude; how different they are, and not which is larger/smaller. The result is another `DataFrame` of the same size as the previous, but now all values are positive.

- `.sum(axis=1)`: this sums the differences across the columns. The result is a `DataFrame` with $34$ rows and a single column with a value that represents how different each of the songs in the dataset is from the input clip. The larger the number, the more different the song is.

What we still have to do is find out which row has the shortest distance to our input clip.

We can do this in a variety of ways.

One possibility is to just loop over the values and use $2$ variables to keep track of the smallest difference value and the index of where that difference is.

A more general approach is to turn our `DataFrame` into a list of pairs of row indexes and difference values:

```py
[[0, difference], [1, difference], [2, difference], ...]
```

and sort by the difference values. As we'll see below, this approach has advantages.

We can turn our single-column `dists` `DataFrame` into a list like this using the `.to_dict()` function followed by the `.items()`.

The `.to_dict()` function turns the single-column `DataFrame` into a `Python` dictionary, like this:
```py
{
  0: difference,
  1: difference,
  2: difference,
  ...
}
```

And the `items()` function is the regular `Python` dictionary that gives us dictionary (`key`,`value`) pairs.

In [ ]:
dists = (tracks_df - in_df.iloc[0]).abs().sum(axis=1)
idx_dist_pairs = dists.to_dict().items()

# break these down into multiple steps and see if each step makes sense
(tracks_df - in_df.iloc[0]) # add abs, sum, to_dict and items one at a time

We still have to sort our distance pairs

In [ ]:
# TODO: sort the idx_dist_pairs list by the difference values
# TODO: after sorting, look at the first item and check if the index matches the index of the input song

### 5. Evaluate

We should calculate how often our Shazam algorithm is right.

This can be done by implementing the following set of steps:
1. Initialize a counter to keep track of number of correct predictions
2. For $N$ predictions (where $N$ can be $10$, $100$, $1000$, etc ...)
3. Pick a random track from the album
4. Pick a random $5$ second clip from the random track
5. Extract features, compare to features in `tracks_df` and predict which track it is
6. If prediction correct, increment correct predictions counter

Alternatively, instead of only incrementing the correct predictions counter when the correct track is the very first on the list of possible tracks, we can increment the counter if the correct track is in the $\text{top-}3$ predictions. This is the $\text{top-}3$ accuracy of our classifier, which, instead of being a binary measurement of right/wrong, also gives us a bit of credit for being "close".

In [ ]:
# TODO: write a function called random5() that
#       receives an audio file path, extracts MFCC features from 
#       a random 5-second portion of the audio and returns a DataFrame with the features

def random5(fpath):
  # TODO: implement here. random.randrange() can help
  return pd.DataFrame.from_records([])

We'll also need a function for comparing the `DataFrame` of songs and the `DataFrame` of the input clip.

We just have to wrap the mega-line of `Pandas` and the sorting from above in a function.

In [ ]:
# TODO: implement comparison function
def compare_predict(tracks_df, in_df):
  # implement here
  return []

Now we use the above defined functions to run some tests.

We'll want to run $200$ to $500$ tests, but we can start with $10$ or so just to make sure the code works.

In [ ]:
# TODO: pick a random song
# TODO: extract features from random 5-second clip of song
# TODO: compare, and keep track of correct predictions
# TODO: repeat
# TODO: print accuracy (number of correct predictions divided by total tests)

### Improve the Algorithm ?

Not bad.

(Re)Watch the Shazam video [here](https://www.youtube.com/watch?v=b6xeOLjeKs0). What's the one big difference between their algorithm and ours ?

(You don't have to improve the code. Just think about what could be some potential next steps.)